# Data Cleaning File

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
df = pd.read_csv('data/strava_activity.csv')

In [3]:
#Making a copy of the data to make changes
clean = df.copy()
clean.columns

Index(['achievement_count', 'athlete.badge_type_id', 'athlete.city',
       'athlete.country', 'athlete.premium', 'athlete.resource_state',
       'athlete.sex', 'athlete.state', 'athlete_count', 'average_cadence',
       'average_heartrate', 'average_speed', 'average_temp', 'average_watts',
       'comment_count', 'commute', 'device_watts', 'distance', 'elapsed_time',
       'elev_high', 'elev_low', 'end_latlng', 'flagged', 'gear_id',
       'has_heartrate', 'has_kudoed', 'kilojoules', 'kudos_count',
       'location_city', 'location_country', 'location_state', 'manual',
       'max_heartrate', 'max_speed', 'max_watts', 'moving_time', 'name',
       'photo_count', 'private', 'resource_state', 'start_date',
       'start_date_local', 'start_latitude', 'start_latlng', 'start_longitude',
       'suffer_score', 'timezone', 'total_elevation_gain', 'total_photo_count',
       'trainer', 'type', 'weighted_average_watts', 'workout_type'],
      dtype='object')

In [4]:
#Dropping all the columns we do not need
clean = clean.drop(['achievement_count', 'average_cadence', 'athlete.badge_type_id', 'athlete.city', 'athlete.country',
                   'athlete.premium', 'athlete.resource_state', 'athlete.state', 'athlete_count', 'comment_count',
                   'device_watts', 'end_latlng', 'flagged', 'gear_id', 'has_kudoed', 'kudos_count', 'name', 'photo_count',
                   'private', 'resource_state', 'start_date', 'start_date_local', 'start_latitude', 'start_latlng',
                   'start_longitude', 'suffer_score', 'total_photo_count', 'trainer', 
                   'workout_type', 'manual', 'location_city', 'location_state'], axis = 1)

In [5]:
#Creating dummy variables
clean = clean[clean['type'].isin(['Ride', 'Run', 'Walk', 'Swim', 'Workout', 'Hike', 'NordicSki'])]
clean = pd.get_dummies(clean, columns = ['type'])

In [6]:
#Removing outliers below 0.1m/s and above 45m/s
clean = clean.loc[(clean.average_speed > 0.1) & (clean.average_speed < 45)]

#Removing unrealistic heartrate values
clean = clean.loc[(clean.average_heartrate != 0) & (clean.max_heartrate != 0) & (clean.max_heartrate < 220)]

#Removing distances =0 , less than 170m and above 80467.2 m
clean = clean.loc[(clean.distance > 170) & (clean.distance < 80467.2)]

#Converting time from seconds to minutes
clean.elapsed_time = clean.elapsed_time / 60
clean.moving_time = clean.moving_time / 60

#Removing time < 5 minutes and > 600 minutes
clean = clean.loc[(clean.elapsed_time > 5) & (clean.elapsed_time < 600)]

#Removing time < 5 minutes and > 600 minutes
clean = clean.loc[(clean.moving_time > 5) & (clean.moving_time < 600)]

#Removing elevation above 4000 m
clean = clean.loc[clean.elev_high < 4000]
clean = clean.loc[clean.elev_low < 4000]

In [7]:
clean.describe()

,average_heartrate,average_speed,average_temp,average_watts,commute,distance,elapsed_time,elev_high,elev_low,has_heartrate,...,moving_time,total_elevation_gain,weighted_average_watts,type_Hike,type_NordicSki,type_Ride,type_Run,type_Swim,type_Walk,type_Workout
count,1523.000000,1523.000000,715.000000,888.000000,1523.000000,1523.000000,1523.000000,1523.000000,1523.000000,1479.0,...,1523.000000,1523.000000,139.000000,1523.000000,1523.000000,1523.000000,1523.000000,1523.000000,1523.000000,1523.000000
mean,140.281944,5.251944,16.829371,143.099550,0.021011,25654.655680,88.713800,300.704859,194.530532,1.0,...,76.008645,241.720683,183.431655,0.002626,0.003283,0.639527,0.344714,0.000657,0.005253,0.003940
std,20.017593,2.331230,7.895074,54.394183,0.143468,20257.415376,63.336299,445.527028,384.144689,0.0,...,46.147729,297.301738,50.407477,0.051198,0.057222,0.480295,0.475431,0.025624,0.072309,0.062663
min,49.000000,0.267000,-3.000000,5.800000,0.000000,311.800000,5.716667,-192.400000,-228.000000,1.0,...,5.266667,0.000000,79.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,129.500000,3.039000,11.000000,109.375000,0.000000,8386.650000,44.716667,49.000000,3.000000,1.0,...,40.516667,30.000000,146.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,142.800000,5.473000,17.000000,139.000000,0.000000,20341.300000,73.000000,144.400000,49.000000,1.0,...,66.300000,127.000000,177.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,153.150000,7.188000,22.000000,170.950000,0.000000,39208.950000,119.166667,329.050000,204.100000,1.0,...,104.591667,362.450000,217.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,187.600000,16.460000,41.000000,506.600000,1.000000,80381.500000,586.383333,3844.200000,2879.400000,1.0,...,311.750000,2632.200000,330.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
#Dropping rows where average speed, distance or elapsed time are NA
clean = clean.dropna(subset = ['average_speed', 'distance', 'elapsed_time', 'moving_time'])
clean = pd.get_dummies(clean, columns = ['athlete.sex'])

In [9]:
clean.location_country = clean.location_country.str.replace("The Netherlands", "Netherlands")

In [10]:
# To Help you see which Countries Need to be cleaned
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.value_counts(clean.location_country).to_frame().reset_index()

,index,location_country
0,United States,415
1,United Kingdom,214
2,Australia,136
3,France,79
4,Spain,69
5,Netherlands,50
6,Italy,48
7,Germany,37
8,Canada,37
9,Brazil,32


In [11]:
#Converting to imperial units
clean['distance_meters'] = clean.apply(lambda row: row.distance * 0.000621371, axis=1)
clean['calories'] = clean.apply(lambda row: row.kilojoules * 0.239006, axis=1)
clean['average_speed_mph'] = clean.apply(lambda row: row.average_speed * 2.23694, axis=1)
clean['max_speed_mph'] = clean.apply(lambda row: row.max_speed * 2.23694, axis=1)

## After Addressing First Question

In [12]:
clean = clean.dropna(subset = ['location_country'])
print(clean.shape)
clean.head()

(1477, 32)


,average_heartrate,average_speed,average_temp,average_watts,commute,distance,elapsed_time,elev_high,elev_low,has_heartrate,kilojoules,location_country,max_heartrate,max_speed,max_watts,moving_time,timezone,total_elevation_gain,weighted_average_watts,type_Hike,type_NordicSki,type_Ride,type_Run,type_Swim,type_Walk,type_Workout,athlete.sex_F,athlete.sex_M,distance_meters,calories,average_speed_mph,max_speed_mph
0,151.2,3.022,NaN,89.8,0,21580.0,144.950000,3000.5,2777.3,1.0,641.4,Ecuador,175.0,10.9,NaN,119.000000,(GMT-05:00) America/Guayaquil,476.3,NaN,0,0,1,0,0,0,0,1,0,13.409186,153.298448,6.760033,24.382646
6,135.9,4.675,6.0,146.0,0,24438.8,89.033333,161.2,99.6,1.0,763.1,Lithuania,182.0,12.8,NaN,87.133333,(GMT+02:00) Europe/Vilnius,303.0,NaN,0,0,1,0,0,0,0,0,1,15.185562,182.385479,10.457695,28.632832
10,143.0,6.685,3.0,221.5,0,33632.8,95.366667,198.0,37.2,1.0,1114.6,United Kingdom,174.0,13.9,NaN,83.850000,(GMT+00:00) Europe/London,301.0,NaN,0,0,1,0,0,0,0,0,1,20.898447,266.396088,14.953944,31.093466
17,150.5,6.124,NaN,95.9,0,28765.1,80.916667,181.5,0.0,1.0,450.6,United Kingdom,174.0,17.1,NaN,78.283333,(GMT+00:00) Europe/London,241.8,NaN,0,0,1,0,0,0,0,0,1,17.873799,107.696104,13.699021,38.251674
18,147.1,9.249,NaN,205.8,0,69559.8,132.433333,378.0,193.3,1.0,1547.7,Australia,178.0,16.8,NaN,125.350000,(GMT+10:00) Australia/Melbourne,383.8,NaN,0,0,1,0,0,0,0,0,1,43.222442,369.909586,20.689458,37.580592


In [13]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.value_counts(clean.location_country).to_frame().reset_index()

,index,location_country
0,United States,415
1,United Kingdom,214
2,Australia,136
3,France,79
4,Spain,69
5,Netherlands,50
6,Italy,48
7,Germany,37
8,Canada,37
9,Brazil,32


In [14]:
clean.to_csv('data/a3clean_data.csv', index = False)